# Cleanup AWS AMIs and instances

This notebook lists and deletes all AMIs and EC2 instances in an AWS account

In [ ]:
import boto3
from datetime import datetime

Change `DELETE` to `True` to delete AMIs and instances

In [ ]:
DELETE = False

In [ ]:
ec2 = boto3.client("ec2")

def list_owned_amis():
    """
    Lists AMIs owned by this AWS account (default is to also list external public AMIs)
    """
    print(datetime.now().isoformat())
    amis = ec2.describe_images(Owners=["self"])
    print(f"{'Name':<50} {'ImageId':<25} {'CreationDate'}")
    for a in sorted(amis["Images"], key=lambda a: a["Name"]):
        print(f"{a['Name']:<50} {a['ImageId']:<25} {a['CreationDate']}")
    return amis["Images"]


def list_instances():
    """
    List AWS instances in this account (includes recently terminated instances)
    """
    print(datetime.now().isoformat())
    instances = ec2.describe_instances()
    print(f"{'Name':<50} {'State':<10} {'InstanceId':<25} {'ImageId':<25} {'LaunchTime'}")
    for r in instances["Reservations"]:
        for i in r["Instances"]:
            name = [t["Value"] for t in i["Tags"] if t["Key"] == "Name"][0]
            print(f"{name:<50} {i['State']['Name']:<10} {i['InstanceId']:<25} {i['ImageId']:<25} {i['LaunchTime']}")
    return [i for r in instances["Reservations"] for i in r["Instances"]]

## AMIs

AMIs owned by this account:

In [ ]:
amis = list_owned_amis()

### Delete AMIs

In [ ]:
if DELETE:
    delete_ami_ids = [a["ImageId"] for a in amis]
else:
    delete_ami_ids = []

for ami in delete_ami_ids:
    r = ec2.deregister_image(ImageId=ami)
    print(r)

In [ ]:
list_owned_amis();

## EC2 instances

This includes instances that were recently terminated

In [ ]:
instances = list_instances()

## Delete instances

In [ ]:
if DELETE:
    delete_instance_ids = [i["InstanceId"] for i in instances]
else:
    delete_instance_ids = []

for instance in delete_instance_ids:
    r = ec2.terminate_instances(InstanceIds=[instance])
    print(r)

In [ ]:
list_instances();